# Embedding + Bidirectional GRU

In [2]:
import os
import time
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


#### Reading data

In [3]:
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
train_df = pd.read_csv(dir_data+'train.csv')
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
test_df = pd.read_csv(dir_data+'test.csv')

#### Preprocessing

Contractions

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

len(contraction_mapping)

162

In [5]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

Lower

In [6]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())
test_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())

Punctuations

In [7]:
punct_mapping = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping += '©^®` <→°€™› ♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'
puncts = {"‘": "'", "´": "'", "°": "", "€": "e", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '…': ' '}

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    return text

In [8]:
train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))
test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))

#### Tokenizing the words for posterior embedding

In [9]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=123)

embed_size = 300 
max_features = 50000 
maxlen = 100 

## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Target
train_y = train_df['target'].values
val_y = val_df['target'].values

## Shuffling data
np.random.seed(42)

trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
train_y = train_y[trn_idx]
val_X = val_X[val_idx]
val_y = val_y[val_idx]

#### Defining a basic model

In [10]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140160    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)  

#### Training

In [11]:
## Train the model 
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1044897 samples, validate on 261225 samples
Epoch 1/2


 104448/1044897 [=>............................] - ETA: 1:35:40 - loss: 0.6997 - acc: 0.29 - ETA: 1:07:16 - loss: 0.6862 - acc: 0.57 - ETA: 58:12 - loss: 0.6741 - acc: 0.6986 - ETA: 52:43 - loss: 0.6622 - acc: 0.76 - ETA: 50:04 - loss: 0.6520 - acc: 0.79 - ETA: 47:49 - loss: 0.6409 - acc: 0.81 - ETA: 46:31 - loss: 0.6299 - acc: 0.83 - ETA: 45:33 - loss: 0.6188 - acc: 0.84 - ETA: 44:33 - loss: 0.6078 - acc: 0.85 - ETA: 43:53 - loss: 0.5953 - acc: 0.86 - ETA: 43:16 - loss: 0.5831 - acc: 0.87 - ETA: 42:55 - loss: 0.5701 - acc: 0.87 - ETA: 42:35 - loss: 0.5579 - acc: 0.88 - ETA: 42:19 - loss: 0.5454 - acc: 0.88 - ETA: 42:02 - loss: 0.5314 - acc: 0.88 - ETA: 41:53 - loss: 0.5180 - acc: 0.89 - ETA: 41:41 - loss: 0.5048 - acc: 0.89 - ETA: 41:37 - loss: 0.4925 - acc: 0.89 - ETA: 41:33 - loss: 0.4793 - acc: 0.89 - ETA: 41:26 - loss: 0.4676 - acc: 0.90 - ETA: 41:24 - loss: 0.4563 - acc: 0.90 - ETA: 41:13 - loss: 0.4470 - acc: 0.90 - ETA: 41:06 - loss: 0.4378 - acc: 0.90 - ETA: 41:01 - loss: 0.42

 208896/1044897 [====>.........................] - ETA: 38:50 - loss: 0.2039 - acc: 0.93 - ETA: 38:49 - loss: 0.2035 - acc: 0.93 - ETA: 38:47 - loss: 0.2031 - acc: 0.93 - ETA: 38:46 - loss: 0.2028 - acc: 0.93 - ETA: 38:44 - loss: 0.2025 - acc: 0.93 - ETA: 38:43 - loss: 0.2022 - acc: 0.93 - ETA: 38:42 - loss: 0.2018 - acc: 0.93 - ETA: 38:41 - loss: 0.2014 - acc: 0.93 - ETA: 38:40 - loss: 0.2010 - acc: 0.93 - ETA: 38:39 - loss: 0.2008 - acc: 0.93 - ETA: 38:38 - loss: 0.2005 - acc: 0.93 - ETA: 38:37 - loss: 0.2001 - acc: 0.93 - ETA: 38:36 - loss: 0.1998 - acc: 0.93 - ETA: 38:35 - loss: 0.1996 - acc: 0.93 - ETA: 38:33 - loss: 0.1993 - acc: 0.93 - ETA: 38:32 - loss: 0.1990 - acc: 0.93 - ETA: 38:31 - loss: 0.1987 - acc: 0.93 - ETA: 38:30 - loss: 0.1985 - acc: 0.93 - ETA: 38:29 - loss: 0.1981 - acc: 0.93 - ETA: 38:28 - loss: 0.1977 - acc: 0.93 - ETA: 38:27 - loss: 0.1974 - acc: 0.93 - ETA: 38:26 - loss: 0.1970 - acc: 0.93 - ETA: 38:25 - loss: 0.1967 - acc: 0.93 - ETA: 38:24 - loss: 0.1965 - a

 313344/1044897 [=======>......................] - ETA: 34:47 - loss: 0.1672 - acc: 0.94 - ETA: 34:46 - loss: 0.1671 - acc: 0.94 - ETA: 34:45 - loss: 0.1670 - acc: 0.94 - ETA: 34:44 - loss: 0.1668 - acc: 0.94 - ETA: 34:43 - loss: 0.1666 - acc: 0.94 - ETA: 34:41 - loss: 0.1666 - acc: 0.94 - ETA: 34:40 - loss: 0.1665 - acc: 0.94 - ETA: 34:39 - loss: 0.1664 - acc: 0.94 - ETA: 34:38 - loss: 0.1663 - acc: 0.94 - ETA: 34:36 - loss: 0.1662 - acc: 0.94 - ETA: 34:35 - loss: 0.1661 - acc: 0.94 - ETA: 34:34 - loss: 0.1661 - acc: 0.94 - ETA: 34:33 - loss: 0.1660 - acc: 0.94 - ETA: 34:31 - loss: 0.1658 - acc: 0.94 - ETA: 34:30 - loss: 0.1657 - acc: 0.94 - ETA: 34:29 - loss: 0.1655 - acc: 0.94 - ETA: 34:28 - loss: 0.1654 - acc: 0.94 - ETA: 34:27 - loss: 0.1653 - acc: 0.94 - ETA: 34:25 - loss: 0.1651 - acc: 0.94 - ETA: 34:24 - loss: 0.1649 - acc: 0.94 - ETA: 34:23 - loss: 0.1648 - acc: 0.94 - ETA: 34:22 - loss: 0.1647 - acc: 0.94 - ETA: 34:20 - loss: 0.1647 - acc: 0.94 - ETA: 34:19 - loss: 0.1646 - a

 417792/1044897 [==========>...................] - ETA: 30:28 - loss: 0.1517 - acc: 0.94 - ETA: 30:27 - loss: 0.1516 - acc: 0.94 - ETA: 30:26 - loss: 0.1516 - acc: 0.94 - ETA: 30:24 - loss: 0.1515 - acc: 0.94 - ETA: 30:23 - loss: 0.1515 - acc: 0.94 - ETA: 30:22 - loss: 0.1515 - acc: 0.94 - ETA: 30:21 - loss: 0.1514 - acc: 0.94 - ETA: 30:19 - loss: 0.1514 - acc: 0.94 - ETA: 30:18 - loss: 0.1514 - acc: 0.94 - ETA: 30:17 - loss: 0.1513 - acc: 0.94 - ETA: 30:16 - loss: 0.1513 - acc: 0.94 - ETA: 30:14 - loss: 0.1512 - acc: 0.94 - ETA: 30:13 - loss: 0.1511 - acc: 0.94 - ETA: 30:12 - loss: 0.1511 - acc: 0.94 - ETA: 30:10 - loss: 0.1511 - acc: 0.94 - ETA: 30:09 - loss: 0.1510 - acc: 0.94 - ETA: 30:08 - loss: 0.1510 - acc: 0.94 - ETA: 30:06 - loss: 0.1509 - acc: 0.94 - ETA: 30:05 - loss: 0.1508 - acc: 0.94 - ETA: 30:04 - loss: 0.1508 - acc: 0.94 - ETA: 30:03 - loss: 0.1507 - acc: 0.94 - ETA: 30:01 - loss: 0.1507 - acc: 0.94 - ETA: 30:00 - loss: 0.1507 - acc: 0.94 - ETA: 29:59 - loss: 0.1507 - a

...

 626688/1044897 [================>.............] - ETA: 22:16 - loss: 0.0988 - acc: 0.96 - ETA: 22:15 - loss: 0.0989 - acc: 0.96 - ETA: 22:14 - loss: 0.0989 - acc: 0.96 - ETA: 22:12 - loss: 0.0989 - acc: 0.96 - ETA: 22:11 - loss: 0.0989 - acc: 0.96 - ETA: 22:10 - loss: 0.0989 - acc: 0.96 - ETA: 22:08 - loss: 0.0989 - acc: 0.96 - ETA: 22:07 - loss: 0.0990 - acc: 0.96 - ETA: 22:06 - loss: 0.0990 - acc: 0.96 - ETA: 22:04 - loss: 0.0990 - acc: 0.96 - ETA: 22:03 - loss: 0.0990 - acc: 0.96 - ETA: 22:02 - loss: 0.0990 - acc: 0.96 - ETA: 22:00 - loss: 0.0990 - acc: 0.96 - ETA: 21:59 - loss: 0.0990 - acc: 0.96 - ETA: 21:58 - loss: 0.0990 - acc: 0.96 - ETA: 21:56 - loss: 0.0990 - acc: 0.96 - ETA: 21:55 - loss: 0.0990 - acc: 0.96 - ETA: 21:54 - loss: 0.0990 - acc: 0.96 - ETA: 21:53 - loss: 0.0989 - acc: 0.96 - ETA: 21:52 - loss: 0.0989 - acc: 0.96 - ETA: 21:51 - loss: 0.0989 - acc: 0.96 - ETA: 21:50 - loss: 0.0989 - acc: 0.96 - ETA: 21:49 - loss: 0.0989 - acc: 0.96 - ETA: 21:47 - loss: 0.0989 - a

 731136/1044897 [===================>..........] - ETA: 17:50 - loss: 0.0985 - acc: 0.96 - ETA: 17:49 - loss: 0.0985 - acc: 0.96 - ETA: 17:47 - loss: 0.0984 - acc: 0.96 - ETA: 17:46 - loss: 0.0985 - acc: 0.96 - ETA: 17:45 - loss: 0.0984 - acc: 0.96 - ETA: 17:43 - loss: 0.0984 - acc: 0.96 - ETA: 17:42 - loss: 0.0984 - acc: 0.96 - ETA: 17:41 - loss: 0.0984 - acc: 0.96 - ETA: 17:39 - loss: 0.0984 - acc: 0.96 - ETA: 17:38 - loss: 0.0984 - acc: 0.96 - ETA: 17:37 - loss: 0.0984 - acc: 0.96 - ETA: 17:36 - loss: 0.0984 - acc: 0.96 - ETA: 17:34 - loss: 0.0984 - acc: 0.96 - ETA: 17:33 - loss: 0.0984 - acc: 0.96 - ETA: 17:32 - loss: 0.0984 - acc: 0.96 - ETA: 17:30 - loss: 0.0984 - acc: 0.96 - ETA: 17:29 - loss: 0.0984 - acc: 0.96 - ETA: 17:28 - loss: 0.0984 - acc: 0.96 - ETA: 17:26 - loss: 0.0984 - acc: 0.96 - ETA: 17:25 - loss: 0.0984 - acc: 0.96 - ETA: 17:24 - loss: 0.0984 - acc: 0.96 - ETA: 17:22 - loss: 0.0984 - acc: 0.96 - ETA: 17:21 - loss: 0.0984 - acc: 0.96 - ETA: 17:20 - loss: 0.0984 - a

 835584/1044897 [======================>.......] - ETA: 13:23 - loss: 0.0986 - acc: 0.96 - ETA: 13:21 - loss: 0.0986 - acc: 0.96 - ETA: 13:20 - loss: 0.0986 - acc: 0.96 - ETA: 13:19 - loss: 0.0986 - acc: 0.96 - ETA: 13:17 - loss: 0.0986 - acc: 0.96 - ETA: 13:16 - loss: 0.0986 - acc: 0.96 - ETA: 13:15 - loss: 0.0986 - acc: 0.96 - ETA: 13:13 - loss: 0.0987 - acc: 0.96 - ETA: 13:12 - loss: 0.0987 - acc: 0.96 - ETA: 13:11 - loss: 0.0987 - acc: 0.96 - ETA: 13:09 - loss: 0.0987 - acc: 0.96 - ETA: 13:08 - loss: 0.0987 - acc: 0.96 - ETA: 13:07 - loss: 0.0986 - acc: 0.96 - ETA: 13:06 - loss: 0.0986 - acc: 0.96 - ETA: 13:04 - loss: 0.0986 - acc: 0.96 - ETA: 13:03 - loss: 0.0986 - acc: 0.96 - ETA: 13:02 - loss: 0.0987 - acc: 0.96 - ETA: 13:00 - loss: 0.0986 - acc: 0.96 - ETA: 12:59 - loss: 0.0986 - acc: 0.96 - ETA: 12:58 - loss: 0.0986 - acc: 0.96 - ETA: 12:56 - loss: 0.0986 - acc: 0.96 - ETA: 12:55 - loss: 0.0986 - acc: 0.96 - ETA: 12:54 - loss: 0.0986 - acc: 0.96 - ETA: 12:52 - loss: 0.0986 - a

 940032/1044897 [=========================>....] - ETA: 8:55 - loss: 0.0988 - acc: 0.960 - ETA: 8:53 - loss: 0.0988 - acc: 0.960 - ETA: 8:52 - loss: 0.0988 - acc: 0.960 - ETA: 8:51 - loss: 0.0988 - acc: 0.960 - ETA: 8:49 - loss: 0.0988 - acc: 0.960 - ETA: 8:48 - loss: 0.0988 - acc: 0.960 - ETA: 8:47 - loss: 0.0988 - acc: 0.960 - ETA: 8:45 - loss: 0.0988 - acc: 0.960 - ETA: 8:44 - loss: 0.0988 - acc: 0.960 - ETA: 8:43 - loss: 0.0988 - acc: 0.960 - ETA: 8:41 - loss: 0.0988 - acc: 0.960 - ETA: 8:40 - loss: 0.0988 - acc: 0.960 - ETA: 8:39 - loss: 0.0988 - acc: 0.960 - ETA: 8:37 - loss: 0.0987 - acc: 0.960 - ETA: 8:36 - loss: 0.0988 - acc: 0.960 - ETA: 8:35 - loss: 0.0988 - acc: 0.960 - ETA: 8:34 - loss: 0.0987 - acc: 0.960 - ETA: 8:32 - loss: 0.0988 - acc: 0.960 - ETA: 8:31 - loss: 0.0987 - acc: 0.960 - ETA: 8:30 - loss: 0.0988 - acc: 0.960 - ETA: 8:28 - loss: 0.0988 - acc: 0.960 - ETA: 8:27 - loss: 0.0988 - acc: 0.960 - ETA: 8:26 - loss: 0.0988 - acc: 0.960 - ETA: 8:24 - loss: 0.0988 - ac

1044897/1044897 [==============================] - ETA: 4:26 - loss: 0.0988 - acc: 0.960 - ETA: 4:25 - loss: 0.0989 - acc: 0.960 - ETA: 4:23 - loss: 0.0988 - acc: 0.960 - ETA: 4:22 - loss: 0.0988 - acc: 0.960 - ETA: 4:21 - loss: 0.0988 - acc: 0.960 - ETA: 4:20 - loss: 0.0988 - acc: 0.960 - ETA: 4:18 - loss: 0.0988 - acc: 0.960 - ETA: 4:17 - loss: 0.0988 - acc: 0.960 - ETA: 4:16 - loss: 0.0988 - acc: 0.960 - ETA: 4:14 - loss: 0.0989 - acc: 0.960 - ETA: 4:13 - loss: 0.0989 - acc: 0.960 - ETA: 4:12 - loss: 0.0989 - acc: 0.960 - ETA: 4:10 - loss: 0.0989 - acc: 0.960 - ETA: 4:09 - loss: 0.0989 - acc: 0.960 - ETA: 4:08 - loss: 0.0989 - acc: 0.960 - ETA: 4:06 - loss: 0.0989 - acc: 0.960 - ETA: 4:05 - loss: 0.0989 - acc: 0.960 - ETA: 4:04 - loss: 0.0989 - acc: 0.960 - ETA: 4:02 - loss: 0.0989 - acc: 0.960 - ETA: 4:01 - loss: 0.0989 - acc: 0.960 - ETA: 4:00 - loss: 0.0989 - acc: 0.960 - ETA: 3:59 - loss: 0.0989 - acc: 0.960 - ETA: 3:57 - loss: 0.0989 - acc: 0.960 - ETA: 3:56 - loss: 0.0989 - ac

In [12]:
model.save_weights("model_Embeddings+GRU.h5")

#### Fscore for validation with different thresholds

In [13]:
pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)

261225/261225 [==============================] - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E

In [14]:
f1_final = 0
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    f1 = metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))
    if f1>f1_final:
        f1_final = f1
        threshold = thresh
print("F1 score at threshold {0} is {1}".format(threshold, metrics.f1_score(val_y, (pred_val_y>threshold).astype(int))))

F1 score at threshold 0.34 is 0.6480105190944431


# Testing with real dataset

In [ ]:
pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
pred_test_y = (pred_test_y>threshold).astype(int)

In [ ]:
submission_df = pd.DataFrame({"qid":test_df["qid"].values})
submission_df['prediction'] = pred_test_y
submission_df.to_csv("submission.csv", index=False)